In [ ]:
# First, install only essential packages
!pip install sentence-transformers faiss-cpu requests beautifulsoup4 numpy

import numpy as np
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import json
import random
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading

# Set up the environment
class ContextAwareChatbot:
    def __init__(self):
        self.vectorstore = None
        self.embeddings_model = None
        self.memory = []
        self.documents = []

    def setup_embeddings(self):
        """Setup embeddings model"""
        try:
            self.embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Embeddings model loaded")
            return True
        except Exception as e:
            print(f"Error loading embeddings: {e}")
            return False

    def load_documents(self, source_type: str, source_input: str):
        """Load documents from various sources"""
        self.documents = []

        try:
            if source_type == "wikipedia":
                print(f"Loading Wikipedia: {source_input}")
                documents = self._load_wikipedia(source_input)

            elif source_type == "web":
                print(f"Loading web page: {source_input}")
                documents = self._load_web_page(source_input)

            elif source_type == "text":
                print("Loading custom text")
                documents = [source_input]

            elif source_type == "sample":
                print("Loading sample knowledge base")
                documents = self._create_sample_documents()

            self.documents = documents
            print(f"Loaded {len(documents)} documents")
            return documents

        except Exception as e:
            print(f"Error loading documents: {e}")
            return self._create_sample_documents()

    def _load_wikipedia(self, query: str):
        """Load content from Wikipedia"""
        try:
            url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get main content
            content = soup.find('div', {'id': 'mw-content-text'})
            if content:
                text = content.get_text()
                return [text[:3000]]  # Limit size
            return ["Wikipedia content not available"]
        except:
            return ["Failed to load Wikipedia content"]

    def _load_web_page(self, url: str):
        """Load content from web page"""
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Remove script and style elements
            for script in soup(["script", "style"]):
                script.decompose()

            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = ' '.join(chunk for chunk in chunks if chunk)

            return [text[:3000]]  # Limit size
        except:
            return ["Web content not available"]

    def _create_sample_documents(self):
        """Create sample documents about AI/ML"""
        sample_docs = [
            """Artificial Intelligence (AI) refers to the simulation of human intelligence in machines.
            AI systems can learn, reason, perceive, and make decisions. Key areas include machine learning,
            natural language processing, computer vision, and robotics.""",

            """Machine Learning is a subset of AI where computers learn from data without explicit programming.
            There are three main types: supervised learning (labeled data), unsupervised learning (unlabeled data),
            and reinforcement learning (reward-based learning).""",

            """Natural Language Processing (NLP) enables computers to understand and generate human language.
            Applications include chatbots, translation, sentiment analysis, and text summarization.
            Modern NLP uses transformer models like BERT and GPT.""",

            """Deep Learning uses neural networks with many layers to analyze complex patterns in data.
            It excels at image recognition, speech recognition, and natural language processing.
            Convolutional Neural Networks (CNNs) are used for images, while RNNs and Transformers handle sequential data.""",

            """Chatbots are AI programs that simulate human conversation. They use NLP to understand user queries
            and generate appropriate responses. Modern chatbots can maintain context and provide personalized assistance."""
        ]
        print("✅ Created sample knowledge base")
        return sample_docs

    def create_vectorstore(self):
        """Create FAISS vector store"""
        if not self.documents or not self.embeddings_model:
            print("❌ No documents or embeddings model")
            return None

        try:
            # Generate embeddings
            embeddings = self.embeddings_model.encode(self.documents)

            # Create FAISS index
            dimension = embeddings.shape[1]
            index = faiss.IndexFlatL2(dimension)
            index.add(np.array(embeddings).astype('float32'))

            self.vectorstore = {
                'index': index,
                'documents': self.documents,
                'embeddings': embeddings
            }

            print("FAISS vector store created")
            return self.vectorstore

        except Exception as e:
            print(f"Error creating vector store: {e}")
            return None

    def retrieve_context(self, question: str, k=3):
        """Retrieve relevant context using semantic search"""
        if not self.vectorstore or not self.embeddings_model:
            return ["No context available"]

        try:
            # Embed the question
            question_embedding = self.embeddings_model.encode([question])

            # Search in vector store
            distances, indices = self.vectorstore['index'].search(
                np.array(question_embedding).astype('float32'), k
            )

            # Get top documents
            results = []
            for i, idx in enumerate(indices[0]):
                if idx < len(self.documents):
                    results.append(self.documents[idx])

            return results if results else [self.documents[0]]

        except Exception as e:
            print(f"Error retrieving context: {e}")
            return [self.documents[0] if self.documents else "No context"]

    def generate_response(self, question: str, context: list):
        """Generate response based on context and memory"""
        # Combine context
        context_text = "\n\n".join(context)[:1000]

        # Check memory for previous context
        memory_context = ""
        if self.memory:
            memory_context = " Previous conversation: " + " ".join(
                [f"Q: {m['question']} A: {m['response']}" for m in self.memory[-2:]]
            )

        # Simple rule-based response generation
        question_lower = question.lower()

        if any(word in question_lower for word in ['hello', 'hi', 'hey']):
            response = "Hello! I'm a context-aware chatbot. How can I help you today?"
        elif any(word in question_lower for word in ['what is', 'what are', 'define']):
            response = f"Based on my knowledge: {context_text[:300]}..."
        elif any(word in question_lower for word in ['how', 'why', 'explain']):
            response = f"Let me explain: {context_text[:300]}..."
        else:
            response = f"I found this information: {context_text[:300]}..."

        return response

    def ask_question(self, question: str):
        """Main method to ask questions"""
        if not self.vectorstore:
            return "Chatbot not initialized. Please setup the knowledge base first."

        try:
            # Retrieve context
            context = self.retrieve_context(question)

            # Generate response
            response = self.generate_response(question, context)

            # Update memory (keep last 5 exchanges)
            self.memory.append({"question": question, "response": response})
            if len(self.memory) > 5:
                self.memory.pop(0)

            # Add context sources
            if context:
                sources = "\n\nSource context:\n" + "\n".join([f"- {doc[:100]}..." for doc in context[:2]])
                response += sources

            return response

        except Exception as e:
            return f"Error: {str(e)}"

    def clear_memory(self):
        """Clear conversation memory"""
        self.memory = []
        print("Memory cleared")
        return True

# Web Interface
class ChatbotHandler(BaseHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        self.chatbot = ContextAwareChatbot()
        # Initialize with sample data
        self.chatbot.setup_embeddings()
        self.chatbot.load_documents("sample", "")
        self.chatbot.create_vectorstore()
        super().__init__(*args, **kwargs)

    def do_GET(self):
        if self.path == '/':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            self.wfile.write(self._get_html().encode())
        else:
            self.send_error(404)

    def do_POST(self):
        if self.path == '/chat':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            data = json.loads(post_data.decode())

            response = self.chatbot.ask_question(data['question'])

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps({'response': response}).encode())
        else:
            self.send_error(404)

    def _get_html(self):
        return """
        <!DOCTYPE html>
        <html>
        <head>
            <title>Context-Aware Chatbot</title>
            <style>
                body {
                    font-family: Arial, sans-serif;
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    min-height: 100vh;
                }
                .container {
                    background: white;
                    border-radius: 15px;
                    padding: 30px;
                    box-shadow: 0 10px 30px rgba(0,0,0,0.2);
                }
                h1 {
                    color: #333;
                    text-align: center;
                    margin-bottom: 30px;
                }
                .chat-container {
                    border: 2px solid #ddd;
                    border-radius: 10px;
                    padding: 20px;
                    height: 400px;
                    overflow-y: auto;
                    margin-bottom: 20px;
                    background: #fafafa;
                }
                .message {
                    margin: 10px 0;
                    padding: 12px;
                    border-radius: 15px;
                    max-width: 80%;
                }
                .user {
                    background: #007bff;
                    color: white;
                    margin-left: auto;
                    text-align: right;
                }
                .bot {
                    background: #e9ecef;
                    color: #333;
                    margin-right: auto;
                }
                input {
                    width: 70%;
                    padding: 12px;
                    border: 2px solid #ddd;
                    border-radius: 25px;
                    margin-right: 10px;
                    font-size: 16px;
                }
                button {
                    padding: 12px 25px;
                    background: #007bff;
                    color: white;
                    border: none;
                    border-radius: 25px;
                    cursor: pointer;
                    font-size: 16px;
                }
                button:hover {
                    background: #0056b3;
                }
                .info {
                    background: #d4edda;
                    padding: 15px;
                    border-radius: 10px;
                    margin: 20px 0;
                }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Context-Aware Chatbot</h1>

                <div class="info">
                    <strong>RAG Chatbot Features:</strong><br>
                    • Retrieval-Augmented Generation<br>
                    • Context Memory (last 5 messages)<br>
                    • Semantic Search with FAISS<br>
                    • AI/ML Knowledge Base
                </div>

                <div class="chat-container" id="chat">
                    <div class="message bot">
                        <strong>Bot:</strong> Hello! I'm a context-aware chatbot with RAG capabilities.
                        Ask me about Artificial Intelligence, Machine Learning, or NLP!
                    </div>
                </div>

                <div style="display: flex;">
                    <input type="text" id="question" placeholder="Ask about AI, ML, NLP..." onkeypress="handleKeyPress(event)">
                    <button onclick="sendQuestion()">Send</button>
                </div>

                <div style="margin-top: 20px; text-align: center;">
                    <button onclick="clearChat()" style="background: #dc3545;">Clear Chat</button>
                </div>
            </div>

            <script>
                function handleKeyPress(e) {
                    if (e.key === 'Enter') sendQuestion();
                }

                function sendQuestion() {
                    const question = document.getElementById('question').value;
                    if (!question) return;

                    // Add user message
                    const chat = document.getElementById('chat');
                    chat.innerHTML += `<div class="message user"><strong>You:</strong> ${question}</div>`;
                    chat.scrollTop = chat.scrollHeight;

                    // Send to server
                    fetch('/chat', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ question: question })
                    })
                    .then(response => response.json())
                    .then(data => {
                        chat.innerHTML += `<div class="message bot"><strong>Bot:</strong> ${data.response}</div>`;
                        chat.scrollTop = chat.scrollHeight;
                    })
                    .catch(error => {
                        chat.innerHTML += `<div class="message bot"><strong>Bot:</strong> Error: ${error}</div>`;
                    });

                    document.getElementById('question').value = '';
                }

                function clearChat() {
                    document.getElementById('chat').innerHTML =
                        '<div class="message bot"><strong>Bot:</strong> Chat cleared! How can I help you?</div>';
                }
            </script>
        </body>
        </html>
        """

def run_web_server():
    """Run the web server"""
    print(" Starting Context-Aware Chatbot Server...")
    print("Server running at: http://localhost:8000")
    print(" Chatbot is ready with AI/ML knowledge base!")
    print(" Try asking: 'What is machine learning?' or 'Explain neural networks'")
    print("  Press Ctrl+C to stop the server")

    server = HTTPServer(('localhost', 8000), ChatbotHandler)
    server.serve_forever()

# Console Interface
def run_console_chat():
    """Run console-based chat"""
    print(" Context-Aware Chatbot with RAG")
    print("=" * 50)

    chatbot = ContextAwareChatbot()
    chatbot.setup_embeddings()
    chatbot.load_documents("sample", "")
    chatbot.create_vectorstore()

    print("\n Chatbot initialized with AI/ML knowledge!")
    print(" Try these questions:")
    print("   - What is artificial intelligence?")
    print("   - Explain machine learning")
    print("   - How does deep learning work?")
    print("   - What are neural networks?")
    print("   - Type 'quit' to exit")
    print("-" * 50)

    while True:
        try:
            question = input("\n You: ").strip()
            if question.lower() in ['quit', 'exit', 'bye']:
                print(" Bot: Goodbye! ")
                break
            if not question:
                continue

            response = chatbot.ask_question(question)
            print(f" Bot: {response}")

        except KeyboardInterrupt:
            print("\n Bot: Goodbye! ")
            break
        except Exception as e:
            print(f" Bot: Error: {e}")

# Main execution
if __name__ == "__main__":
    print(" Context-Aware RAG Chatbot")
    print("Choose interface:")
    print("1.  Web Interface (Browser)")
    print("2.  Console Interface")

    choice = input("Enter choice (1 or 2): ").strip()

    if choice == "1":
        run_web_server()
    else:
        run_console_chat()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.4 MB/s eta 0:00:00
🚀 Context-Aware RAG Chatbot
Choose interface:
1. 🌐 Web Interface (Browser)
2. 💻 Console Interface
🌐 Starting Context-Aware Chatbot Server...
📍 Server running at: http://localhost:8000
🤖 Chatbot is ready with AI/ML knowledge base!
💡 Try asking: 'What is machine learning?' or 'Explain neural networks'
⏹️  Press Ctrl+C to stop the server
